# データソースが複数ある場合のオフラインRLについて

参考
* [Provably Efficient Offline Reinforcement Learning with Perturbed Data Sources](https://arxiv.org/abs/2306.08364)

通常のオフライン強化学習の理論ではデータの出所が単一ですが，アプリケーションではそのような状況は稀です．
今回はデータが複数のソースからくる場合のオフライン強化学習について見てみます．（論文のFigure 1参照）

表記
* 通常のエピソディックMDPを考えます．初期分布を$\xi \in \Delta(\mathcal{S})$として，$V_1^{\pi, \mathcal{M}}(\xi):=\mathbb{E}_{s \sim \xi}\left[V_1^{\pi, \mathcal{M}}(s)\right]$と書きます．
* Occupancy measureは$d_h^{\pi, \mathcal{M}}(s ; \xi):=\mathbb{E}_{\pi, \mathcal{M}}\left[\mathbb{1}\left\{s_h=s\right\} \mid s_1 \sim \xi\right]$と書きます．
* Suboptimality gapを$\operatorname{Gap}(\hat{\pi} ; \mathcal{M}, \xi):=V_1^{\pi^*, \mathcal{M}}(\xi)-V_1^{\hat{\pi}, \mathcal{M}}(\xi)$とします．
* $L$個の異なるソースからデータがやってくる設定を考えます．$l$個目のデータソースを$\mathcal{M}_l=\left(H, \mathcal{S}, \mathcal{A}, \mathbb{P}_l, r_l\right)$とします．ターゲットタスクは$\mathcal{M}$です．また，データソースのタスクのMDPとターゲットタスクのMDPには次の関係が成立することを仮定します：

---

データソースのMDP $\left\{\mathcal{M}_l=\left\{H, \mathcal{S}, \mathcal{A}, \mathbb{P}_l, r_l\right\}: l \in[L]\right\}$は未知の分布$g=\left\{g_h: h \in[H]\right\}$から，それぞれの$(l, h) \in[L] \times[H]$について，独立してサンプルされるとする．ここで，そのサンプルの期待値は$\left\{r_h, \mathbb{P}_h\right\}$になると仮定する．

---

問題設定
* それぞれのデータソースから，データセット$\mathcal{D}_l:=\left\{\left(s_{1, l}^k, a_{1, l}^k, r_{1, l}^k, \cdots, s_{H, l}^k, a_{H, l}^k, r_{H, l}^k\right): k \in[K]\right\}$がサンプルされてくるとします．
    * このようにして作られたデータセットは，各$l$内のデータは相関があることに注意しましょう．
    * これはTwo-fold sub-samplingテクニックで回避できます：[Settling the Sample Complexity of Model-Based Offline Reinforcement Learning](https://arxiv.org/abs/2204.05275)など参照．
    * Two-fold sub-samplingで再構成されたデータセットを$\left\{\mathcal{D}_l^{\prime}: l \in[L]\right\}$と書くことにします．
* $\mathcal{L}_h(s, a):=\left\{l \in[L]: d_h^{\rho_l, \mathcal{M}_l}\left(s, a ; \xi_l\right)>0\right\} \subseteq[L]$を，$(s, a, h)$を訪問可能なデータセットのインデックスの集合とします．
    * 最適方策が訪問する$(s, a, h)$のデータソースの最小の数の集合を$L^{\dagger}:=\min \left\{\left|\mathcal{L}_h(s, a)\right|:(s, a, h)\right.$ s.t. $\left.d_h^{\pi^*, \mathcal{M}}(s, a ; \xi)>0\right\}$
        * 最適方策の情報を与えるデータソースの数を表してます．
    * Coverageのパラメータを$C^{\dagger}:=\max _{(s, a, h)}\left\{\sum_{l \in \mathcal{L}_h(s, a)} \frac{\min \left\{d_h^{\pi^*, \mathcal{M}^{\prime}}(s, a ; \xi), \frac{1}{S}\right\}}{\left|\mathcal{L}_h(s, a)\right| \cdot d_h^{\rho_l, \mathcal{M}_l}\left(s, a ; \xi_l\right)}\right\}$とします．$0/0=0$とします．
        * データソースがどれくらい良い情報を与えているかを表しています

この$L^{\dagger}$と$C^{\dagger}$ですが，それぞれ一定以上大きくなければ，$\varepsilon$-最適な方策の学習が絶対にできないMDPが存在します．（詳しくは論文のTheorem 3.1参照）

## アルゴリズム

次のアルゴリズムを考えます：

1. 入力：データセット$\mathcal{D}=\left\{D_l: l \in[L]\right\}$
2. $\mathcal{D}_l^{\prime} \leftarrow \operatorname{subsampling}\left(\mathcal{D}_l\right), \forall l \in[L]$
3. データセットを使って，報酬と遷移，そしてそれぞれの状態行動のデータソースの数を推定します：
   1. データソースの数：$\hat{\mathcal{L}}_h(s, a):=\left\{l \in [L]: N_{h, l}(s, a)>0\right\}$． そのサイズ：$\hat{L}_h(s, a)$
4. ペナルティとQ値を計算し，動的計画法をします．
$$
\begin{aligned}
& \Gamma_h^\alpha(s, a) \leftarrow c \sqrt{\sum_{l \in \hat{\mathcal{L}}_h(s, a)} \frac{H^2 \log (S A H / \delta)}{\left(\hat{L}_h(s, a)\right)^2 N_{h, l}(s, a)}} \\
& \Gamma_h^\beta(s, a) \leftarrow c \sqrt{H^2 \log (S A H / \delta) / \hat{L}_h(s, a)} \\
& \Gamma_h(s, a) \leftarrow \min \left\{\Gamma_h^\alpha(s, a)+\Gamma_h^\beta(s, a), H\right\} \\
& \hat{Q}_h(s, a) \leftarrow \max \left\{\left(\hat{\mathbb{B}}_h \hat{V}_{h+1}\right)(s, a)-\Gamma_h(s, a), 0\right\}
\end{aligned}
$$

---

**ペナルティについて**

今回はサンプルについての不確かさだけでなく，データソースについての不確かさも考えなければなりません．

サンプルについての不確かさを：
$$\Gamma_h^\alpha(s, a)=c \sqrt{\frac{1}{\left(\hat{L}_h(s, a)\right)^2} \sum_{l \in \hat{\mathcal{L}}_h(s, a)} \frac{H^2 \log (S A H / \delta)}{N_{h, l}(s, a)}}$$

データソースについての不確かさを：
$$
\Gamma_h^\beta(s, a)=c \sqrt{\frac{H^2 \log (S A H / \delta)}{\hat{L}_h(s, a)}} .
$$
とします．
そしてペナルティを

$$
\Gamma_h(s, a)=\min \left\{\Gamma_h^\alpha(s, a)+\Gamma_h^\beta(s, a), H\right\},
$$

として表します．これにより，高確率で

$$\left|\left(\hat{\mathbb{B}}_h \hat{V}_{h+1}\right)(s, a)-\left(\mathbb{B}_h \hat{V}_{h+1}\right)(s, a)\right| \leq \Gamma_h(s, a)$$

が保証できます．

---

**コメント** 途中でロバストRLについての応用も話しているが，Uncertainty setのパラメータ$\sigma$は既知っぽい．
